In [ ]:
# This script gets all the unique album IDs
import os
import json

folder_path = 'C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\artist_albums\\dak'
unique_ids = set() 

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):  
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            for item in data:
                unique_ids.add(item['id']) 

output_file_path = os.path.join(folder_path, 'unique_ids.txt')
with open(output_file_path, 'w') as output_file:
    for id in unique_ids:
        output_file.write(id + '\n')

print(f"Unique IDs have been written to {output_file_path}")

In [ ]:
# This script merges the album IDs from the smaller lists, removing duplicates
def load_list_from_file(file_path):
    try:
        with open(file_path, 'r') as file:
            # Reads each line into a list, stripping newline characters
            return [line.strip() for line in file]
    except FileNotFoundError:
        return []

file_path1 = 'dak_unique_ids.txt'
file_path2 = 'rory_unique_ids.txt'
file_path3 = 'will_unique_ids.txt'
list1 = load_list_from_file(file_path1)
list2 = load_list_from_file(file_path2)
list3 = load_list_from_file(file_path3)
merged_list = list(set(list1 + list2 + list3))
output_file_path = 'merged_artist_ids_list.txt'
with open(output_file_path, 'w') as output_file:
    for id in merged_list:
        output_file.write(id + '\n') 

print(merged_list)

In [ ]:
# This script calls the Spotify API to get the album details for each album ID in batches of 20
import requests
import json
import os
import time

def load_album_ids(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print("File not found.")
        return []

def get_albums_info(album_ids, access_token, market='US'):
    url = 'https://api.spotify.com/v1/albums'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'ids': ','.join(album_ids), 'market': market}
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(response.headers)
        print(f"Failed to retrieve albums information. Status code: {response.status_code}, Response: {response.text}")
        return None

# Your Spotify access token and file paths
access_token = 'BQBwbcsu_xDB_--wEKxV6fhQsASgvhYmx7tQIwajQlisM8sBuRMffAFK1-so7TOy3IXwk_OJsdcqlMU-cYFPW74-9DEl4BGi2JqlgytuP2GEaR2Kv_RP9uAZOnQBOXrxEjNKcaX7pPfmdbe_qmO6ZAlC8qtRW7g8ZuYbYUAx1XUVZUS529AyFe3LcU9c2QJC0eVcmJc7fVE07L5GbFpe6DG45WEkZvUgbpQ4boJFv0sGjub2xLXVtm9REvShQnY_9dd0s7vZNdUSOW-73cqM78sBh12jsSaCsN0P6IiD'  # Replace with your actual Spotify access token
album_ids_file_path = 'merged_artist_ids_list.txt'  # The path to your file containing album IDs
output_directory = 'album_information'
os.makedirs(output_directory, exist_ok=True)

# Load the album IDs
album_ids = load_album_ids(album_ids_file_path)

# Process the list in chunks of 20 IDs at a time
for i in range(0, len(album_ids), 20):
    chunk = album_ids[i:i+20]
    albums_info = get_albums_info(chunk, access_token)
    if albums_info:
        # Consider timestamp or some unique attribute for filename uniqueness if required
        timestamp = int(time.time())
        with open(f'{output_directory}/albums_info_{i}-{i+len(chunk)}_{timestamp}.json', 'w') as output_file:
            json.dump(albums_info, output_file)

print("Albums information has been saved.")
